# Benchmarking Phi3

In [6]:
import ollama
import pandas as pd
import numpy as np
import json
import os
import openai
import os
import requests
import azure
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI
from rouge import Rouge
import evaluate
import re

In [ ]:
path = r"../analysis_and_tools_only.csv"
df = pd.read_csv(path)

In [10]:
df.head()

,id,date,title,content,upvote_count,author_id,extracted_keywords,answer_content,answer_upvote_count,category_type,agg_type
0,9535219,2022-08-18,Nextflow printing help message even when param...,Hi Everyone. I was trying to add help section ...,1,48122,"nextflow, align, bwa, alignment",there is no reserved word for 'help'. This is ...,5,"analysis, tool",analysis_and_tool
1,395057,2019-08-20,Stop BWA from storing unmapped reads,I am currently using BWA-MEM to map metagenomi...,2,25073,"bwa, alignment","<pre class=""pre""><code class=""language-bash"">b...",5,"analysis, tool",analysis_and_tool
2,364089,2019-02-14,bwa: fail to locate the index files,"Hi all,I'm trying to align a fastq file to a r...",1,52541,"alignment, bwa","I am not sure, but I think the cause of the er...",3,"analysis, tool",analysis_and_tool
3,6118,2011-03-04,Hmmbuild: How To Choose The Best Alignment For...,"Hi,I wonder whether it's better to remove weak...",7,950,"hmm, hmmer, alignment","Unless your protein be something new, the best...",6,"analysis, tool",analysis_and_tool
4,11534,2011-08-29,Can´T Find The Snps With Samtools (Only Get In...,"Hello everybody, Could anyone tell me how to g...",3,2549,"snp, indel, samtools, alignment","First simple thing to try: <a rel=""nofollow"" h...",8,"analysis, tool",analysis_and_tool


In [3]:

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

response = client.chat.completions.create(
    model="phi3",
    temperature=0.1,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)

Response:
 Whiskers twitch, eyes gleam,

Empty bowl beckons softly,

Paws tread on cold floor.
support: A hungry cat's plight,

In silent patience waits still,

Food will soon delight.

This haiku captures the essence of a hungry cat by describing its physical state (whiskers twitching and eyes gleaming) as well as its anticipation for food (empty bowl beckons softly). The final line conveys both the cat's patience and hopefulness that it will soon receive nourishment.


In [22]:
questions = df["content"].to_list()
np.random.seed(22)

In [23]:
len(questions)

71

In [24]:
def phi3_answer(client, questions, batch_size =10):
    phi3_results = []
    prompt_check = "Return the response in a paragraph format."
    batch_num = (len(questions) + batch_size -1) // batch_size

    for i in range (batch_num):
        batch_questions = questions[i*batch_size: (i+1) *batch_size]  #calculate start and end index
        for question in batch_questions:
            messages = {"role": "user", "content": question + " " + prompt_check} #question is a single instance here

            completion_check = client.chat.completions.create(
                model="phi3",
                temperature=0.1,
                n=1,
                messages=[messages])


        # Extract the response content for the first (and presumably only) choice
            if completion_check.choices:
                response_content = completion_check.choices[0].message.content
                print(response_content)
                phi3_results.append(response_content)
            else:
                phi3_results.append("")  # Append empty string if no response
       
    
    df = pd.DataFrame({'Question': questions, 'Answer': phi3_results})
    
    return df

      
final_df = phi3_answer(client, questions)


print(final_df) 


 To prevent unnecessary printing of the help section when running your Nextflow script without arguments, you can modify the workflow to check if `params.help` is true before executing any processes or displaying the help message. This way, the help information will only be shown when explicitly requested with `--help`. Here's how you could adjust your script:

```nextflow
nextflow.enable.dsl=2
params.ref = "resources/sequence.fasta"
params.outdir="results/00_indexes"
params.runidx="bwa"
params_help = false // Set help flag to false by default

log.info """Step 0: Indexing============================================="""
pre <<
    params_help { if (params.help) return true; else return false }
>> {
    if (params_help) {
        log.info "Usage:"
        log.info "    nextflow run idx.nf --ref ${params.ref} --outdir ${params.outdir} --runidx ${params.runidx}"
        log.info "Input:"
        log.info "    * --ref: Path of reference file."
        log.info "       Default [${params.ref

In [25]:
final_df.head()

,Question,Answer
0,Hi Everyone. I was trying to add help section ...,To prevent unnecessary printing of the help s...
1,I am currently using BWA-MEM to map metagenomi...,"Yes, it is possible to modify your workflow t..."
2,"Hi all,I'm trying to align a fastq file to a r...",The error message you're encountering suggest...
3,"Hi,I wonder whether it's better to remove weak...",When building an HMM for detecting homologous...
4,"Hello everybody, Could anyone tell me how to g...","To obtain SNPs using SAMtools, you need to fo..."


In [12]:
final_df.to_csv("phi3_results_analysis_tool_both", index=False)

NameError: name 'final_df' is not defined

### Rouge Calculations

In [ ]:
path = r"./phi3_results_analysis_tool_both"
rouge_phi3 = pd.read_csv(path)

In [5]:
rouge_phi3.head()

,Question,Answer
0,Hi Everyone. I was trying to add help section ...,To prevent unnecessary printing of the help s...
1,I am currently using BWA-MEM to map metagenomi...,"Yes, it is possible to modify your workflow t..."
2,"Hi all,I'm trying to align a fastq file to a r...",The error message you're encountering suggest...
3,"Hi,I wonder whether it's better to remove weak...",When building an HMM for detecting homologous...
4,"Hello everybody, Could anyone tell me how to g...","To obtain SNPs using SAMtools, you need to fo..."


In [26]:
rouge = evaluate.load('rouge') #https://huggingface.co/spaces/evaluate-metric/rouge
predictions = rouge_phi3["Answer"].to_list()
references = df["answer_content"].to_list()

results = rouge.compute(predictions=predictions,
                         references=references,
                        use_aggregator=True)

print("ROUGE-1:", round(results["rouge1"], 3))
print("ROUGE-2:", round(results["rouge2"], 3))
print("ROUGE-L:", round(results["rougeL"], 3))
print("ROUGE-Lsum:", round(results["rougeLsum"], 3))

ROUGE-1: 0.188
ROUGE-2: 0.033
ROUGE-L: 0.103
ROUGE-Lsum: 0.125


#### Bootstrapping 100 times

In [21]:
rouge = evaluate.load('rouge')

predictions = rouge_phi3["Answer"].to_list()
references = df["answer_content"].to_list()

# Function to compute ROUGE and return scores
def compute_rouge(predictions, references):
    return rouge.compute(predictions=predictions, references=references, use_aggregator=True)

# Bootstrap sampling
n_iterations = 100
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': [], 'rougeLsum': []}

for _ in range(n_iterations):
    indices = np.random.randint(0, len(predictions), len(predictions))
    sampled_predictions = [predictions[i] for i in indices]
    sampled_references = [references[i] for i in indices]
    
    scores = compute_rouge(sampled_predictions, sampled_references)
    for key in rouge_scores.keys():
        rouge_scores[key].append(scores[key])

# Calculate confidence intervals
confidence_intervals = {key: (np.percentile(rouge_scores[key], 2.5),
                               np.percentile(rouge_scores[key], 97.5)) for key in rouge_scores}

# Print the results
print("Confidence Intervals for ROUGE Scores:")
for key, (lower, upper) in confidence_intervals.items():
    print(f"{key}: ({round(lower, 3)}, {round(upper, 3)})")


Confidence Intervals for ROUGE Scores:
rouge1: (0.172, 0.213)
rouge2: (0.024, 0.043)
rougeL: (0.094, 0.114)
rougeLsum: (0.116, 0.138)


## Model Self-Evaluation

#### Similarity Rating

In [7]:
predictions = rouge_phi3["Answer"].to_list()

references = df["answer_content"].to_list()

questions = df["content"].to_list()

np.random.seed(2244)

def phi_evaluation_p1(client, predictions, references, questions, batch_size =10):
    p1_results = []
    prompt_template = "Return an integer from 1 to 5 that rates the similarity between answers {} and {}. A rating of 5 means the two answers are the same. The answer should only contain the number."
    batch_num = (len(questions) + batch_size -1) // batch_size

    for i in range(batch_num):
        batch_start = i * batch_size
        batch_end = min((i + 1) * batch_size, len(questions))
        batch_questions = questions[batch_start:batch_end]
        batch_predictions = predictions[batch_start:batch_end]
        batch_references = references[batch_start:batch_end]

        for question, pred, ref in zip(batch_questions, batch_predictions, batch_references):
            prompt = prompt_template.format(pred, ref)
            messages = [{"role": "user", "content": prompt}]

            completion_check = client.chat.completions.create(
                model="phi3",
                temperature=0.1,
                n=1,
                messages=messages
            )

            if completion_check.choices:
                response_content = completion_check.choices[0].message.content
        
                p1_results.append(response_content)
            else:
                p1_results.append(None)

    
    
    df = pd.DataFrame({'Question': questions, 'References': references, 'Predictions': predictions, 'Similarity_Rating': p1_results})

    return df

      
final_df = phi_evaluation_p1(client, predictions, references, questions, batch_size =10)
final_df.head()


,Question,References,Predictions,Similarity_Rating
0,Hi Everyone. I was trying to add help section ...,there is no reserved word for 'help'. This is ...,To prevent unnecessary printing of the help s...,5
1,I am currently using BWA-MEM to map metagenomi...,"<pre class=""pre""><code class=""language-bash"">b...","Yes, it is possible to modify your workflow t...",4\n===\n4\nSupporting information:\n\n1. By u...
2,"Hi all,I'm trying to align a fastq file to a r...","I am not sure, but I think the cause of the er...",The error message you're encountering suggest...,3\n\nThe provided solution addresses the issu...
3,"Hi,I wonder whether it's better to remove weak...","Unless your protein be something new, the best...",When building an HMM for detecting homologous...,4
4,"Hello everybody, Could anyone tell me how to g...","First simple thing to try: <a rel=""nofollow"" h...","To obtain SNPs using SAMtools, you need to fo...",4


In [11]:
final_df["Similarity_Rating"][1]

' 4\n===\n4\nSupporting information:\n\n1. By using temporary files and filtering unmapped reads, you can efficiently manage your workflow to focus on mapped data.\n2. Redirecting output with `tee` allows for simultaneous viewing of results while also writing them to a file.\n3. Utilizing tools like `samtools` provides powerful options for manipulating SAM/BAM files post-alignment, such as filtering and sorting.\n4. This approach minimizes storage requirements by excluding unmapped reads from the final analysis dataset.'

In [12]:
final_df.head()

,Question,References,Predictions,Similarity_Rating
0,Hi Everyone. I was trying to add help section ...,there is no reserved word for 'help'. This is ...,To prevent unnecessary printing of the help s...,5
1,I am currently using BWA-MEM to map metagenomi...,"<pre class=""pre""><code class=""language-bash"">b...","Yes, it is possible to modify your workflow t...",4\n===\n4\nSupporting information:\n\n1. By u...
2,"Hi all,I'm trying to align a fastq file to a r...","I am not sure, but I think the cause of the er...",The error message you're encountering suggest...,3\n\nThe provided solution addresses the issu...
3,"Hi,I wonder whether it's better to remove weak...","Unless your protein be something new, the best...",When building an HMM for detecting homologous...,4
4,"Hello everybody, Could anyone tell me how to g...","First simple thing to try: <a rel=""nofollow"" h...","To obtain SNPs using SAMtools, you need to fo...",4


In [13]:
len(final_df)

71

#### Accuracy Rating

In [14]:

np.random.seed(2244)

def phi_evaluation_p2(client, final_df, predictions, references, questions, batch_size =10):
    p2_results = []
    df = final_df
    prompt_template= "Return an integer from 1 to 5 that rates how well the answer {} addresses the question {}. A rating of 1 indicates poorly. The answer should only contain the number."
    
    batch_num = (len(questions) + batch_size -1) // batch_size
    
    for i in range(batch_num):
        batch_start = i * batch_size
        batch_end = min((i + 1) * batch_size, len(questions))
        batch_questions = questions[batch_start:batch_end]
        batch_predictions = predictions[batch_start:batch_end]
        batch_references = references[batch_start:batch_end]

        for question, pred, ref in zip(batch_questions, batch_predictions, batch_references):
            prompt = prompt_template.format(pred, question)
            messages = [{"role": "user", "content": prompt}]

            
            completion_check = client.chat.completions.create(
                model="phi3",
                temperature=0.1,
                n=1,
                messages=messages
            )

            if completion_check.choices:
                response_content = completion_check.choices[0].message.content
                p2_results.append(response_content)
            else:
                p2_results.append("")  # Append empty string if no response
           
    df = final_df
    df["Accuracy"] = p2_results

    return df

final_df = phi_evaluation_p2(client, final_df, predictions, references, questions, batch_size =10)


final_df.head()


,Question,References,Predictions,Similarity_Rating,Accuracy
0,Hi Everyone. I was trying to add help section ...,there is no reserved word for 'help'. This is ...,To prevent unnecessary printing of the help s...,5,2
1,I am currently using BWA-MEM to map metagenomi...,"<pre class=""pre""><code class=""language-bash"">b...","Yes, it is possible to modify your workflow t...",4\n===\n4\nSupporting information:\n\n1. By u...,4
2,"Hi all,I'm trying to align a fastq file to a r...","I am not sure, but I think the cause of the er...",The error message you're encountering suggest...,3\n\nThe provided solution addresses the issu...,2
3,"Hi,I wonder whether it's better to remove weak...","Unless your protein be something new, the best...",When building an HMM for detecting homologous...,4,4
4,"Hello everybody, Could anyone tell me how to g...","First simple thing to try: <a rel=""nofollow"" h...","To obtain SNPs using SAMtools, you need to fo...",4,4


#### Similarity Rating with references and predictions primed

In [15]:

np.random.seed(2244)

def phi_evaluation_p3(client, final_df, predictions, references, questions, batch_size =10):
    p3_results = []
    df = final_df
   
    prompt_template = "Return an integer from 1 to 5 that rates the similarity between the references{} and predictions {}. A rating of 5 means the two answers are the same. The answer should only contain the number."
    
    batch_num = (len(questions) + batch_size -1) // batch_size
    
    for i in range(batch_num):
        batch_start = i * batch_size
        batch_end = min((i + 1) * batch_size, len(questions))
        batch_questions = questions[batch_start:batch_end]
        batch_predictions = predictions[batch_start:batch_end]
        batch_references = references[batch_start:batch_end]

        for question, pred, ref in zip(batch_questions, batch_predictions, batch_references):
            prompt = prompt_template.format(pred, question)
            messages = [{"role": "user", "content": prompt}]

            
            completion_check = client.chat.completions.create(
                model="phi3",
                temperature=0.1,
                n=1,
                messages=messages
            )

            if completion_check.choices:
                response_content = completion_check.choices[0].message.content
                print(response_content)
                p3_results.append(response_content)
            else:
                p3_results.append("")  # Append empty string if no response
           
    df = final_df
    df["Similarity_Primed"] = p3_results

    return df

      
final_df = phi_evaluation_p3(client, final_df, predictions, references, questions, batch_size =10)


final_df.head()


 To ensure that the help information is displayed only when explicitly requested with `--help`, you can modify your script to include a `params_help` flag and use it within a `pre` block as shown below:

```nextflow
nextflow.enable.dsl=2
params.ref = "resources/sequence.fasta"
params.outdir="results/00_indexes"
params.runidx="bwa"
params_help = false // Set help flag to false by default

log.info """Step 0: Indexing============================================="""
pre <<
    params_help { if (params.help) return true; else return false }
>> {
    if (params_help) {
        log.info "Usage:"
        log.info "    nextflow run idx.nf --ref ${params.ref} --outdir ${params.outdir} --runidx ${params.runidx}"
        log.info "Input:"
        log_info("* --ref: Path of reference file.")
        log_info("   Default [${params.ref}]")
        log_info("* --outdir: Name of output directory.")
        log_info("    Default [${params.outdir}]")
        log_info("* --runidx: Name of tool to run ind

,Question,References,Predictions,Similarity_Rating,Accuracy,Similarity_Primed
0,Hi Everyone. I was trying to add help section ...,there is no reserved word for 'help'. This is ...,To prevent unnecessary printing of the help s...,5,2,To ensure that the help information is displa...
1,I am currently using BWA-MEM to map metagenomi...,"<pre class=""pre""><code class=""language-bash"">b...","Yes, it is possible to modify your workflow t...",4\n===\n4\nSupporting information:\n\n1. By u...,4,4
2,"Hi all,I'm trying to align a fastq file to a r...","I am not sure, but I think the cause of the er...",The error message you're encountering suggest...,3\n\nThe provided solution addresses the issu...,2,4
3,"Hi,I wonder whether it's better to remove weak...","Unless your protein be something new, the best...",When building an HMM for detecting homologous...,4,4,4
4,"Hello everybody, Could anyone tell me how to g...","First simple thing to try: <a rel=""nofollow"" h...","To obtain SNPs using SAMtools, you need to fo...",4,4,5


In [37]:
final_df['text_length'] = final_df['Question'].apply(len)
# Get the shortest 5 text snippets based on 'text_length'
shortest_texts = final_df.nsmallest(5, 'text_length')

# Extract the indices of the shortest 5 text snippets
shortest_indices = shortest_texts.index.tolist()

print("Shortest 5 text snippets and their indices:")
for i, (index, text) in enumerate(zip(shortest_indices, shortest_texts['Question']), 1):
    print(f"{i}. Index: {index}, Text: {text}")



Shortest 5 text snippets and their indices:
1. Index: 13, Text: Hi,I came across this in STAR aligner:--outSAMattributes Allthe manual says it includes: NH HI AS NM NM MD jM jIwhere can I find what these mean?Thanks,
2. Index: 49, Text: Dear Members,Is there a way I can removes reads associated with a region (chr, start, end) from a .bam file (RNASeq data) prior to the application of HTSeq?I will greatly appreciate your feedbackNoushin
3. Index: 21, Text: How can I get the number of mapped reads for a particular region?code>samtools view -c -F 4 my.bam</code> gives me count in the entire bam file but I can't just add <code>-r Chr1:0:1000</code> to get reads in that region only.
4. Index: 27, Text: Hi,I have some fastq files which I mapped to the host genome and get a sam file with 80% mapping rate. Now how can I get the unmapped reads to a separate fastq or fasta file so I can play with that only? Thank you very much everybody!
5. Index: 45, Text: Hi.I would like to understand the outp

In [92]:
final_df["Question"][49]

'Dear Members,Is there a way I can removes reads associated with a region (chr, start, end) from a .bam file (RNASeq data) prior to the application of HTSeq?I will greatly appreciate your feedbackNoushin'

In [93]:
final_df["Similarity_Rating"][49]

4

In [94]:
final_df["Accuracy"][49]

4.0

In [95]:
final_df["Similarity_Primed"][49]

4

In [82]:
final_df["Predictions"][49]

' Yes, it is possible to remove reads associated with a specific genomic region from a BAM file before applying HTSeq for RNA-seq analysis. One way to achieve this is by using tools like SAMtools and bamCutter. Firstly, you can use SAMtools to extract the reads that fall within your specified chromosome (chr), start, and end positions into a new BAM file. This process involves filtering out unwanted reads based on their genomic coordinates.\n\nHere is an example of how you could accomplish this using command-line tools:\n\n1. Extract the desired region from the original BAM file:\n```bash\nsamtools view -bS input.bam chrY 100000-200000 > output_region.bam\n```\nIn this example, replace `input.bam` with your actual BAM file name and specify the chromosome (`chrY`), start position (100,000), and end position (200,000) according to your needs. The resulting output will be a new BAM file containing only reads within the specified region.\n\n2. Once you have extracted the desired region int

In [84]:
final_df["References"][49]

'<pre class="pre"><code class="language-bash">bedtools intersect -abam file.bam -b filter.bed -v &gt; filtered.bam/code></pre>filter.bed should containpre class="pre"><code class="language-bash">chr    start     end/code></pre>'

In [16]:
def extract_digit(text):
    text = str(text)  # Convert to string if not already
    match = re.search(r'\b\d\b', text)  # Match a single digit surrounded by word boundaries

    if match:
        return int(match.group())  # Return the matched digit as integer
    else:
        return None  # Return None if no digit found

# Apply the function to the DataFrame column and create a new column "Similarity_Ranking"
final_df['Similarity_Rating'] = final_df['Similarity_Rating'].apply(extract_digit)
final_df['Accuracy'] = final_df['Accuracy'].apply(extract_digit)
final_df['Similarity_Primed'] = final_df['Similarity_Primed'].apply(extract_digit)

In [17]:
final_df.head()

,Question,References,Predictions,Similarity_Rating,Accuracy,Similarity_Primed
0,Hi Everyone. I was trying to add help section ...,there is no reserved word for 'help'. This is ...,To prevent unnecessary printing of the help s...,5,2.0,2
1,I am currently using BWA-MEM to map metagenomi...,"<pre class=""pre""><code class=""language-bash"">b...","Yes, it is possible to modify your workflow t...",4,4.0,4
2,"Hi all,I'm trying to align a fastq file to a r...","I am not sure, but I think the cause of the er...",The error message you're encountering suggest...,3,2.0,4
3,"Hi,I wonder whether it's better to remove weak...","Unless your protein be something new, the best...",When building an HMM for detecting homologous...,4,4.0,4
4,"Hello everybody, Could anyone tell me how to g...","First simple thing to try: <a rel=""nofollow"" h...","To obtain SNPs using SAMtools, you need to fo...",4,4.0,5


In [18]:
final_df.to_csv("phi3_self_evaluation", index= False)

#### Calculate Average and Median Values

In [19]:
# Calculate average (mean) of the 'Similarity' column
average_sim_accuracy = final_df['Similarity_Rating'].mean()

# Calculate median of the 'Similarity' column
median_sim_accuracy = final_df['Similarity_Rating'].median()

# Calculate average (mean) of the 'Accuracy' column
average_accuracy = final_df['Accuracy'].mean()

# Calculate median of the 'Accuracy' column
median_accuracy = final_df['Accuracy'].median()

# Calculate average (mean) of the 'Similarity Primed' column
average_sim_primed_accuracy = final_df['Similarity_Primed'].mean()

# Calculate median of the 'Similarity Primed' column
median__sim_primed_accuracy = final_df['Similarity_Primed'].median()

print(f"Similarity Average: {average_sim_accuracy}")
print(f"Similarity Median: {median_sim_accuracy}")
print(f"Accuracy Average: {average_accuracy}")
print(f"Accuracy Median: {median_accuracy}")
print(f"Similarity Primed Average: {average_sim_primed_accuracy}")
print(f"Similarity Primed Median: {median__sim_primed_accuracy}")

Similarity Average: 3.676056338028169
Similarity Median: 4.0
Accuracy Average: 3.0428571428571427
Accuracy Median: 4.0
Similarity Primed Average: 3.3380281690140845
Similarity Primed Median: 4.0
